In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

import gc

pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)

Populating the interactive namespace from numpy and matplotlib


In [2]:
DBS_PATH = "../Bases/"

In [3]:
df_processed = pd.read_csv(DBS_PATH + "processed_train_games.csv", index_col=0)

In [4]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10029 entries, 0 to 10028
Columns: 3906 entries, rated to winner
dtypes: bool(595), float64(332), int64(2977), object(2)
memory usage: 259.1+ MB


In [5]:
df_processed.head(2)

,rated,turns,white_rating,black_rating,moves,opening_ply,victory_status_draw,victory_status_mate,victory_status_outoftime,victory_status_resign,opening_eco_A00,opening_eco_A16,opening_eco_A28,opening_eco_A40,opening_eco_A41,opening_eco_A43,opening_eco_A46,opening_eco_A48,opening_eco_B00,opening_eco_B01,opening_eco_B10,opening_eco_B12,opening_eco_B15,opening_eco_B17,opening_eco_B18,opening_eco_B20,opening_eco_B21,opening_eco_B30,opening_eco_B45,opening_eco_B52,opening_eco_B54,opening_eco_B56,opening_eco_B93,opening_eco_B94,opening_eco_C00,opening_eco_C02,opening_eco_C11,opening_eco_C13,opening_eco_C20,opening_eco_C21,opening_eco_C22,opening_eco_C26,opening_eco_C34,opening_eco_C40,opening_eco_C41,opening_eco_C42,opening_eco_C44,opening_eco_C45,opening_eco_C46,opening_eco_C50,opening_eco_C51,opening_eco_C53,opening_eco_C55,opening_eco_C58,opening_eco_C61,opening_eco_C67,opening_eco_C70,opening_eco_C74,opening_eco_C78,opening_eco_D00,opening_eco_D01,opening_eco_D02,opening_eco_D05,opening_eco_D06,opening_eco_D07,opening_eco_D12,opening_eco_D20,opening_eco_E90,duration_in_seconds,increment_code_minutes,increment_code_seconds,rating_difference,#P_moves_white_until_move1,#center_P_moves_white_until_move1,#R_moves_white_until_move1,#N_moves_white_until_move1,#B_moves_white_until_move1,#Q_moves_white_until_move1,#K_moves_white_until_move1,castled_kingside_white_until_move1,castled_queenside_white_until_move1,castled_white_until_move1,#takes_white_until_move1,#checks_white_until_move1,#king_sided_moves_white_until_move1,#queen_sided_moves_white_until_move1,#centered_moves_white_until_move1,#upper_sided_moves_white_until_move1,#lower_sided_moves_white_until_move1,#middle_moves_white_until_move1,#P_moves_black_until_move1,#center_P_moves_black_until_move1,#R_moves_black_until_move1,#N_moves_black_until_move1,#B_moves_black_until_move1,#Q_moves_black_until_move1,#K_moves_black_until_move1,castled_kingside_black_until_move1,castled_queenside_black_until_move1,castled_black_until_move1,#takes_black_until_move1,#checks_black_until_move1,#king_sided_moves_black_until_move1,#queen_sided_moves_black_until_move1,#centered_moves_black_until_move1,#upper_sided_moves_black_until_move1,#lower_sided_moves_black_until_move1,#middle_moves_black_until_move1,#P_moves_white_until_move2,#center_P_moves_white_until_move2,#R_moves_white_until_move2,#N_moves_white_until_move2,#B_moves_white_until_move2,#Q_moves_white_until_move2,#K_moves_white_until_move2,castled_kingside_white_until_move2,castled_queenside_white_until_move2,castled_white_until_move2,#takes_white_until_move2,#checks_white_until_move2,#king_sided_moves_white_until_move2,#queen_sided_moves_white_until_move2,#centered_moves_white_until_move2,#upper_sided_moves_white_until_move2,#lower_sided_moves_white_until_move2,#middle_moves_white_until_move2,#P_moves_black_until_move2,#center_P_moves_black_until_move2,#R_moves_black_until_move2,#N_moves_black_until_move2,#B_moves_black_until_move2,#Q_moves_black_until_move2,#K_moves_black_until_move2,castled_kingside_black_until_move2,castled_queenside_black_until_move2,castled_black_until_move2,#takes_black_until_move2,#checks_black_until_move2,#king_sided_moves_black_until_move2,#queen_sided_moves_black_until_move2,#centered_moves_black_until_move2,#upper_sided_moves_black_until_move2,#lower_sided_moves_black_until_move2,#middle_moves_black_until_move2,#P_moves_white_until_move3,#center_P_moves_white_until_move3,#R_moves_white_until_move3,#N_moves_white_until_move3,#B_moves_white_until_move3,#Q_moves_white_until_move3,#K_moves_white_until_move3,castled_kingside_white_until_move3,castled_queenside_white_until_move3,castled_white_until_move3,#takes_white_until_move3,#checks_white_until_move3,#king_sided_moves_white_until_move3,#queen_sided_moves_white_until_move3,#centered_moves_white_until_move3,#upper_sided_moves_white_until_move3,#lower_sided_moves_white_until_move3,#middle_moves_white_until_move3,#P_moves_black_until_move3,#center_P_moves_black_until_move

Drop the moves

In [6]:
df_processed = df_processed.drop(columns=["moves"])

Convert the boolean columns to int

In [7]:
bool_columns = df_processed.select_dtypes(include=['bool']).columns

In [8]:
df_processed[bool_columns] = df_processed[bool_columns].apply(lambda x : x.astype(int))

## Feature Selection

We save the target

In [9]:
y_backup = df_processed["winner"].copy()

Now, for each value of target value, let's apply Boruta and select which features are more important for each case

- https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a
- https://github.com/scikit-learn-contrib/boruta_py
- https://www.youtube.com/watch?v=pBb0BATEim8
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel

In [10]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

In [11]:
import joblib

In [12]:
class FeatureSelection:
    def __init__(self):
        self.selector = None
        self.strong_features = []
        self.weak_features = []
        
        model_rf = RandomForestClassifier(
            n_jobs=-1, 
            max_depth=5,
            random_state=42
        )
        self.selector = BorutaPy(
            estimator=model_rf, 
            n_estimators='auto',
            max_iter=100, # number of trials to perform
            random_state=42
        )
    
    def fit(self, X, y):
        ### fit Boruta (it accepts np.array, not pd.DataFrame)
        self.selector.fit(np.array(X), np.array(y))
        self.strong_features = X.columns[self.selector.support_].to_list()
        self.weak_features = X.columns[self.selector.support_weak_].to_list()
        # show summary
        self.show_selected_features(X)
        self.show_rank(X)
        
    def transform(self, X, y=None):
        return X[self.strong_features]
        
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X, y)
    
    ############## Auxiliary fuctions ##################
    
    def show_selected_features(self, X):
        print('Feature Selection Summary')
        print('='*40)
        print("# Total of selected features: {}/{}".format(len(self.strong_features), X.shape[0]))
        print('## Strong features - {}'.format(len(self.strong_features)))
        print('## Weak features - {}'.format(len(self.weak_features)))
        print()
        print('# Strong features list')
        df_strong_features = pd.DataFrame()
        df_strong_features["Column"] = self.strong_features
        print(df_strong_features.sort_values(by=["Column"]))
        print()
        print('# Weak features list')
        df_weak_features = pd.DataFrame()
        df_weak_features["Column"] = self.weak_features
        print(df_weak_features.sort_values(by=["Column"]))
        print()
        
    def show_rank(self, X):
        df_ranking = pd.DataFrame()
        df_ranking["Column"] = X.columns
        df_ranking["Ranking"] = self.selector.ranking_
        print(df_ranking.sort_values(by=["Ranking"]))
        print()

### Target: White winner

We recharge the original target

In [13]:
df_processed["winner"] = y_backup.copy()

We apply the new transformation to the target

In [14]:
df_processed["winner"] = df_processed["winner"].apply(lambda x : 1 if (x == "white") else 0)

In [15]:
X_train = df_processed.drop(columns=["winner"])
y_train = df_processed["winner"].copy()

In [16]:
fs_white = FeatureSelection()

In [17]:
X_selected_white = fs_white.fit_transform(X_train, y_train)

Feature Selection Summary
# Total of selected features: 508/10029
## Strong features - 508
## Weak features - 1

# Strong features list
                                    Column
8              #K_moves_black_until_move11
11             #K_moves_black_until_move12
17             #K_moves_black_until_move13
23             #K_moves_black_until_move14
31             #K_moves_black_until_move15
39             #K_moves_black_until_move16
47             #K_moves_black_until_move17
56             #K_moves_black_until_move18
65             #K_moves_black_until_move19
75             #K_moves_black_until_move20
85             #K_moves_black_until_move21
95             #K_moves_black_until_move22
105            #K_moves_black_until_move23
115            #K_moves_black_until_move24
125            #K_moves_black_until_move25
135            #K_moves_black_until_move26
145            #K_moves_black_until_move27
155            #K_moves_black_until_move28
165            #K_moves_black_until_move29
175 

In [18]:
joblib.dump(fs_white.selector, "boruta_white.joblib");
joblib.dump(fs_white.strong_features, "strong_features_white.joblib");
joblib.dump(fs_white.weak_features, "weak_features_white.joblib");

Selected variables for white winning:

- Quantity of King moves at the beginning and middle game for both players
- Quantity of checks at the beginning and middle game for both players
- Quantity of takes at the beginning and middle game for both players
- Quantity of lower sided moves at the beginning and middle game for both players
- Quantity of upper sided moves at the beginning and middle game for both players
- Rating difference

The rating difference is reasonable, as it's related with players level.

The quantity of King moves is reasonable, as at the beginning the development is important.

The quantity of takes make sense, as it's related to material.

### Target: Black winner

We recharge the original target

In [19]:
df_processed["winner"] = y_backup.copy()

We apply the new transformation to the target

In [20]:
df_processed["winner"] = df_processed["winner"].apply(lambda x : 1 if (x == "black") else 0)

In [21]:
X_train = df_processed.drop(columns=["winner"])
y_train = df_processed["winner"].copy()

In [22]:
fs_black = FeatureSelection()

In [23]:
X_selected_black = fs_black.fit_transform(X_train, y_train)

Feature Selection Summary
# Total of selected features: 537/10029
## Strong features - 537
## Weak features - 2

# Strong features list
                                    Column
14             #K_moves_black_until_move13
21             #K_moves_black_until_move14
29             #K_moves_black_until_move15
37             #K_moves_black_until_move16
45             #K_moves_black_until_move17
54             #K_moves_black_until_move18
63             #K_moves_black_until_move19
73             #K_moves_black_until_move20
84             #K_moves_black_until_move21
94             #K_moves_black_until_move22
104            #K_moves_black_until_move23
115            #K_moves_black_until_move24
125            #K_moves_black_until_move25
135            #K_moves_black_until_move26
145            #K_moves_black_until_move27
155            #K_moves_black_until_move28
165            #K_moves_black_until_move29
175            #K_moves_black_until_move30
185            #K_moves_black_until_move31
195 

In [24]:
joblib.dump(fs_black.selector, "boruta_black.joblib");
joblib.dump(fs_black.strong_features, "strong_features_black.joblib");
joblib.dump(fs_black.weak_features, "weak_features_black.joblib");

Selected variables for black winning (almost the same as white):

- Quantity of King moves at the beginning and middle game for both players
- Quantity of checks at the beginning and middle game for both players
- Quantity of takes at the beginning and middle game for both players
- Quantity of lower sided moves at the beginning and middle game for both players
- Quantity of upper sided moves at the beginning and middle game for both players
- Rating difference

The rating difference is reasonable, as it's related with players level.

The quantity of King moves is reasonable, as at the beginning the development is important.

The quantity of takes make sense, as it's related to material.

### Target: Draw

We recharge the original target

In [25]:
df_processed["winner"] = y_backup.copy()

We apply the new transformation to the target

In [26]:
df_processed["winner"] = df_processed["winner"].apply(lambda x : 1 if (x == "draw") else 0)

In [27]:
X_train = df_processed.drop(columns=["winner"])
y_train = df_processed["winner"].copy()

In [28]:
fs_draw = FeatureSelection()

In [29]:
X_selected_draw = fs_draw.fit_transform(X_train, y_train)

Feature Selection Summary
# Total of selected features: 12/10029
## Strong features - 12
## Weak features - 10

# Strong features list
                                   Column
5             #K_moves_white_until_move48
8             #K_moves_white_until_move49
7             #N_moves_white_until_move49
9    #king_sided_moves_white_until_move49
10  #lower_sided_moves_white_until_move49
6   #queen_sided_moves_black_until_move48
11  #queen_sided_moves_black_until_move49
0                                   turns
2                     victory_status_draw
3                     victory_status_mate
4                   victory_status_resign
1                            white_rating

# Weak features list
                                  Column
3            #K_moves_black_until_move47
6            #K_moves_black_until_move48
0            #Q_moves_white_until_move38
4   #king_sided_moves_white_until_move48
8  #lower_sided_moves_black_until_move48
5  #lower_sided_moves_white_until_move48
1  #queen_

In [30]:
joblib.dump(fs_draw.selector, "boruta_draw.joblib");
joblib.dump(fs_draw.strong_features, "strong_features_draw.joblib");
joblib.dump(fs_draw.weak_features, "weak_features_draw.joblib");

Selected variables for white winning:

- King moves 48 and 49 (24th movement)

It's interesting to see the variables #king_sided_moves_white_until_move49, #lower_sided_moves_white_until_move49, #queen_sided_moves_black_until_move48, #queen_sided_moves_black_until_move49

At the same time, we see that the dataset is really unbalanced for this value of y.

---

### Observation

We can see in all the possible values that the quantity of `turns` is important. That means that by having the quantity of turns, it may be determinant for a model to guess the result of the match.

We could validate the hypothesis of the i,portance of that varible being related or not to the level of players, once we see that the range of ratings are placed in a large interval (we see that the 3rd quartil is 1793 for white and 1784 for black, meaning that at least 75% of the players in the dataset are not top players).

At the same time, one 